In [ ]:
'''
Dataset Info.

1. train.csv

train 데이터 : 2019년 01월 01일부터 2023년 03월 03일까지의 유통된 품목의 가격 데이터
item: 품목 코드
TG : 감귤
BC : 브로콜리
RD : 무
CR : 당근
CB : 양배추
corporation : 유통 법인 코드
법인 A부터 F 존재
location : 지역 코드
J : 제주도 제주시
S : 제주도 서귀포시
supply(kg) : 유통된 물량, kg 단위
price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위


2. international_trade.csv

관련 품목 수출입 정보


3. test.csv

test 데이터 : 2023년 03월 04일부터 2023년 03월 31일까지의 데이터


4. sample_submission.csv

제출을 위한 양식
2023년 03월 04일부터 2023년 03월 31일까지의 price(원/kg)을 예측
ID는 품목, 유통 법인, 지역 코드로 구성된 식별자
해당 ID에 맞춰 price(원/kg) 예측값을 answer 컬럼에 기입해야 함
'''

In [1]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/jeju/train.csv')
test = pd.read_csv('/content/drive/MyDrive/jeju/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/jeju/sample_submission.csv')
inter = pd.read_csv('/content/drive/MyDrive/jeju/international_trade.csv')

In [2]:
inter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1274 entries, 0 to 1273
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기간      1274 non-null   object
 1   품목명     1274 non-null   object
 2   수출 중량   1274 non-null   int64 
 3   수출 금액   1274 non-null   int64 
 4   수입 중량   1274 non-null   int64 
 5   수입 금액   1274 non-null   int64 
 6   무역수지    1274 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 69.8+ KB


In [3]:
inter.head()

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
4,2019-01,방울다다기 양배추,0,0,7580,38,-38


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           59397 non-null  object 
 1   timestamp    59397 non-null  object 
 2   item         59397 non-null  object 
 3   corporation  59397 non-null  object 
 4   location     59397 non-null  object 
 5   supply(kg)   59397 non-null  float64
 6   price(원/kg)  59397 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.2+ MB


In [5]:
train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [6]:
train[train['item']=='BC']

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
42644,BC_A_J_20190101,2019-01-01,BC,A,J,0.0,0.0
42645,BC_A_J_20190102,2019-01-02,BC,A,J,0.0,0.0
42646,BC_A_J_20190103,2019-01-03,BC,A,J,7616.0,2559.0
42647,BC_A_J_20190104,2019-01-04,BC,A,J,7488.0,2425.0
42648,BC_A_J_20190105,2019-01-05,BC,A,J,10408.0,2097.0
...,...,...,...,...,...,...,...
56346,BC_E_S_20230227,2023-02-27,BC,E,S,2200.0,2488.0
56347,BC_E_S_20230228,2023-02-28,BC,E,S,1024.0,3232.0
56348,BC_E_S_20230301,2023-03-01,BC,E,S,2160.0,3816.0
56349,BC_E_S_20230302,2023-03-02,BC,E,S,1152.0,3321.0


In [9]:
print('TG' ,len(train[train['item']=='TG'])) #10
print('BC' ,len(train[train['item']=='BC'])) #9
print('RD' ,len(train[train['item']=='RD'])) #8
print('CR' ,len(train[train['item']=='CR'])) #7
print('CB' ,len(train[train['item']=='CB'])) #5

TG 15230
BC 13707
RD 12184
CR 10661
CB 7615


In [19]:
from datetime import datetime

# Define the two dates
date1 = datetime(2019, 1, 1)
date2 = datetime(2023, 3, 3)

# Calculate the difference
date_difference = date2 - date1

# Extract the number of days
date_difference  # 1522+1 = 1523

datetime.timedelta(days=1522)

In [26]:
sub.head()

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
